Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from icecream import ic
import heapq
from collections import deque

In [2]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

## Helper Functions

In [6]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions


#change the position of 0
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def manhattan_distance(state: np.ndarray, final_state: np.ndarray) -> int:
    dist = 0
    for value in range(1, PUZZLE_DIM**2):
        x1, y1 = np.where(state == value)  # current position
        x2, y2 = np.where(final_state == value)  # desired position
        # conversion to scalars
        x1, y1 = int(x1[0]), int(y1[0])
        x2, y2 = int(x2[0]), int(y2[0])
        dist += abs(x1 - x2) + abs(y1 - y2)
    return dist

#for each move avayable we check if that move can lower the manhattan distance. if yes we repeat the process until we can't improve anymore
def greedy_start(state: np.ndarray, final_state: np.ndarray, best_distance: int) -> np.ndarray:
    best_state = state.copy()
    bestdistance = best_distance
    for action in available_actions(best_state):
        curr_state = state.copy()
        new_state = do_action(curr_state, action)
        new_distance = manhattan_distance(new_state, final_state)
        if(new_distance < best_distance):
            curr_state = greedy_start(new_state, final_state, new_distance)
        
        curr_distance = manhattan_distance(curr_state, final_state)
        if(curr_distance < bestdistance):
            best_state = curr_state
            bestdistance = curr_distance
    
    return best_state

#A star algorithm
def a_star(initial_state: np.ndarray, final_state: np.ndarray, cost: int) -> list:
    open_set = []
    initial_state_tuple = tuple(initial_state.flatten())
    final_state_tuple = tuple(final_state.flatten())
    
    #priority queue ordered by f(n)
    heapq.heappush(open_set, (0, initial_state_tuple))  # (f(n), state)
    came_from = {}  # keep track of parents to build the path
    g_scores = {initial_state_tuple: 0}  # Minimum cost to reach each state
    #precalculation of the position of each number in the final state
    goal_positions = {value: (i, j) for i in range(PUZZLE_DIM) for j in range(PUZZLE_DIM) for value in [final_state[i, j]]} #dictionary with each value associated to his pair of coordinates

    def manhattan_distance(state: tuple) -> int:
        #It calculates manhattan distance using pre caluclated position (saving in computational time)
        state_array = np.array(state).reshape(PUZZLE_DIM, PUZZLE_DIM)
        dist = 0
        for i in range(PUZZLE_DIM):
            for j in range(PUZZLE_DIM):
                value = state_array[i, j]
                if value != 0:
                    x_goal, y_goal = goal_positions[value]
                    dist += abs(i - x_goal) + abs(j - y_goal)
        return dist

    while open_set:
        _, current_state_tuple = heapq.heappop(open_set)
        
        # Check if it's the solution
        if current_state_tuple == final_state_tuple:
            path = []
            current = final_state_tuple
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return [np.array(state).reshape(PUZZLE_DIM, PUZZLE_DIM) for state in path], cost

        current_state = np.array(current_state_tuple).reshape(PUZZLE_DIM, PUZZLE_DIM)
        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            new_state_tuple = tuple(new_state.flatten())
            
            #cost to reach new state
            tentative_g_score = g_scores[current_state_tuple] + 1
            if new_state_tuple not in g_scores or tentative_g_score < g_scores[new_state_tuple]: #checks if we have never reached that state or if we can reach it with less moves
                g_scores[new_state_tuple] = tentative_g_score
                #f(n) = g(n) + h(n)
                f_score = tentative_g_score + manhattan_distance(new_state_tuple)
                cost = cost + 1
                heapq.heappush(open_set, (f_score, new_state_tuple))
                came_from[new_state_tuple] = current_state_tuple

    return None  # no solution found


## initialization of a random problem

In [11]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
finalstate = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

initialstate = state

state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[ 6,  4,  8,  5],
       [15,  9,  2, 14],
       [11, 10,  0,  7],
       [ 3,  1, 13, 12]])

## A star

In [12]:
cost = 1
distance = manhattan_distance(initialstate, finalstate)
greedystate = greedy_start(initialstate, finalstate, distance)
solution, cost = a_star(greedystate, finalstate, cost)
print(f"Solution found in {len(solution)} moves:")
for step in solution:
    print(step)
print(cost)

Solution found in 50 moves:
[[ 6  4  2  8]
 [15  9  7  5]
 [11 10 14  0]
 [ 3  1 13 12]]
[[ 6  4  2  8]
 [15  9  7  5]
 [11 10  0 14]
 [ 3  1 13 12]]
[[ 6  4  2  8]
 [15  9  7  5]
 [11  0 10 14]
 [ 3  1 13 12]]
[[ 6  4  2  8]
 [15  9  7  5]
 [11  1 10 14]
 [ 3  0 13 12]]
[[ 6  4  2  8]
 [15  9  7  5]
 [11  1 10 14]
 [ 3 13  0 12]]
[[ 6  4  2  8]
 [15  9  7  5]
 [11  1  0 14]
 [ 3 13 10 12]]
[[ 6  4  2  8]
 [15  9  0  5]
 [11  1  7 14]
 [ 3 13 10 12]]
[[ 6  4  0  8]
 [15  9  2  5]
 [11  1  7 14]
 [ 3 13 10 12]]
[[ 6  0  4  8]
 [15  9  2  5]
 [11  1  7 14]
 [ 3 13 10 12]]
[[ 6  9  4  8]
 [15  0  2  5]
 [11  1  7 14]
 [ 3 13 10 12]]
[[ 6  9  4  8]
 [15  1  2  5]
 [11  0  7 14]
 [ 3 13 10 12]]
[[ 6  9  4  8]
 [15  1  2  5]
 [ 0 11  7 14]
 [ 3 13 10 12]]
[[ 6  9  4  8]
 [15  1  2  5]
 [ 3 11  7 14]
 [ 0 13 10 12]]
[[ 6  9  4  8]
 [15  1  2  5]
 [ 3 11  7 14]
 [13  0 10 12]]
[[ 6  9  4  8]
 [15  1  2  5]
 [ 3  0  7 14]
 [13 11 10 12]]
[[ 6  9  4  8]
 [15  1  2  5]
 [ 0  3  7 14]
 [13 11 10 1